In [ ]:
import gc
import os
import cv2
import math
import numpy as np
import pandas as pd
from functools import partial
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from cloud_tpu_client import Client

IMAGE_SIZE = 224

In [ ]:
c = Client()
c.configure_tpu_version(tf.__version__, restart_type="always")

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
test  = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")

In [ ]:
# Thanks to https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96

def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

In [ ]:
X = np.array([ 
        cv2.resize(
            cv2.cvtColor(
                cv2.imread(f"../input/petfinder-pawpularity-score/train/{idx}.jpg"), 
                cv2.COLOR_BGR2RGB
            ),
            (IMAGE_SIZE, IMAGE_SIZE)
        ) 
        for idx in train["Id"] 
    ], 
    dtype=np.uint8
)
y = (np.array(train["Pawpularity"], dtype=np.float32) - 1) / 99
y_raw = np.array(train["Pawpularity"], dtype=np.float32)

In [ ]:
row = 4
col = 4
for (img,label) in tf.data.Dataset.from_tensor_slices((X[:1], y[:1])).repeat().map(transform).batch(row*col):
    plt.figure(figsize=(15,int(15*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break

In [ ]:
X_test = np.array([ cv2.resize(cv2.cvtColor(cv2.imread(f"../input/petfinder-pawpularity-score/test/{idx}.jpg"), cv2.COLOR_BGR2RGB), (IMAGE_SIZE, IMAGE_SIZE)) for idx in test["Id"] ], dtype=np.uint8)

len(X_test)

In [ ]:
gc.collect()

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print("Running on TPU ", tpu.cluster_spec().as_dict()["worker"])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
EPOCHS = 330
LR_START = 0.00001
LR_MAX = 0.0001 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 16
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .99

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

rng = [i for i in range(EPOCHS)]
plt.plot(rng, [lrfn(x) for x in rng])
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(min([lrfn(x) for x in rng]), max([lrfn(x) for x in rng]), [lrfn(x) for x in rng][-1]))

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["rsme"])
    plt.plot(hist.history["val_rsme"])
    plt.title("model accuracy")
    plt.ylabel("rsme")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
BATCH_SIZE = 8192

# create dictionaries to store predictions
oof_preds = np.zeros((len(y),))
test_preds = np.zeros((len(X_test),))

# create cv
kf = KFold(n_splits=5, shuffle=True, random_state=1)

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    # create train, validation sets
    X_train, y_train = X[idx_train], y[idx_train]
    X_valid, y_valid = X[idx_valid], y[idx_valid]
    
    train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat(32)\
                .map(transform, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                .cache()\
                .repeat()\
                .shuffle(16384, seed=42)\
                .batch(BATCH_SIZE)\
                .prefetch(tf.data.experimental.AUTOTUNE)
    
    with strategy.scope():
        inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
        
        effnet = EfficientNetB0(include_top=False, weights=None) # load weights from dataset (no internet)
        effnet.load_weights("../input/effnetsb0-b7-notops/efficientnetb0_notop.h5")
        effnet.trainable = True

        x = effnet(inputs)

        # top
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(1)(x)

        model = tf.keras.Model(inputs, x)
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-3),
            loss="mae",
            metrics=[keras.metrics.RootMeanSquaredError(name="rsme")]
        )
    
    hist = model.fit(
        train_ds, 
        validation_data=(X_valid, y_valid),
        batch_size=BATCH_SIZE,
        steps_per_epoch = len(y_train) // BATCH_SIZE + 1 if len(y_train) % BATCH_SIZE > 0 else 0,
        callbacks = [
            tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False), 
            #tf.keras.callbacks.ModelCheckpoint( # ????
            #    filepath="/tmp/checkpoint",
            #    save_weights_only=True,
            #    monitor="val_rsme",
            #    mode="min",
            #    save_best_only=True,
            #    experimental_io_device="/job:localhost"
            #)
        ],
        epochs=EPOCHS
    )
    model.save_weights(f"./weights_{fold}.h5", overwrite=True)
    
    plot_hist(hist)
    
    #model.load_weights("/tmp/checkpoint")
    
    oof_preds[idx_valid] = model.predict(X_valid, batch_size=1024).reshape(-1) * 99 + 1
    test_preds += model.predict(X_test, batch_size=1024).reshape(-1) * 99 + 1
    
    print(f"RSME: {math.sqrt(mean_squared_error(y_raw[idx_valid], oof_preds[idx_valid]))}")

print(f"Overall RSME: {math.sqrt(mean_squared_error(y_raw, oof_preds))}")

In [ ]:
sample_submission = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
sample_submission["Pawpularity"] = test_preds
sample_submission.to_csv("submission.csv")
sample_submission.head()